In [9]:
#from __future__ import annotations

from PIL import Image
from pyrr import Matrix44

import moderngl as mgl
import numpy as np

import pygame
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import serial
import serial.tools.list_ports
import time
import math 

start = time.time()

pygame.init()
screen = pygame.display.set_mode([1920, 1080], pygame.RESIZABLE)
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 32)
font2 = pygame.font.SysFont("Arial", 64)

fpslist = []
def pilImageToSurface(pilImage):
    return pygame.image.fromstring(
        pilImage.tobytes(), pilImage.size, pilImage.mode).convert()

        
def show_fbo(fbo: mgl.Framebuffer, size: tuple[int], color_mode: str) -> None:
    img = Image.frombytes(color_mode, size, fbo.read(components=len(color_mode)))
    img = img.transpose(Image.FLIP_TOP_BOTTOM)
    pygameSurface = pilImageToSurface(img)
    #screen.fill(0)
    screen.blit(pygameSurface, pygameSurface.get_rect(center = (1400, 490)))

def update_fps():
	fps = int(clock.get_fps())
	fpslist.append(fps)
	fpstring = str(fps)
	fps_text = font.render(fpstring, 1, pygame.Color("coral"))
	return fps_text

def portsDetection():
     print("waiting for connection...")
     ports = list(serial.tools.list_ports.comports())
     for p in ports:
          print(p.manufacturer)
          if "Arduino" in p.manufacturer:
               print("This is an Arduino!")
               return(serial.Serial(str(p.device), 115200, timeout=10))
     raise Exception("No device Found")

def button(screen, position, text):
	font = pygame.font.SysFont("Arial", 50)
	text_render = font.render(text, 1, (255, 0, 0))
	x, y, w , h = text_render.get_rect()
	x, y = position
	pygame.draw.line(screen, (150, 150, 150), (x, y), (x + w , y), 5)
	pygame.draw.line(screen, (150, 150, 150), (x, y - 2), (x, y + h), 5)
	pygame.draw.line(screen, (50, 50, 50), (x, y + h), (x + w , y + h), 5)
	pygame.draw.line(screen, (50, 50, 50), (x + w , y+h), [x + w , y], 5)
	pygame.draw.rect(screen, (100, 100, 100), (x, y, w , h))
	return screen.blit(text_render, (x, y))


ser = "none"    
SIZE = WIDTH, HEIGHT = int(1080), int(1080)
COLOR_MODE = "RGBA"
ctx = mgl.create_context(standalone=True)
fbo = ctx.simple_framebuffer(SIZE, components=len(COLOR_MODE))
fbo.use()

vbo = ctx.buffer(np.array([
    # x     y    z     u    v  
     1,  1, 0.0,  1., 1.,
     1, -1, 0.0,  1., -1.,
    -1, -1, 0.0,  -1., -1.,
    -1,  1, 0.0,  -1., 1.,
], dtype=np.float32))

ibo = ctx.buffer(np.array([
    0, 1, 3,
    1, 2, 3,
], dtype=np.int32))

b1 = button(screen, (200, 700), "Auto")
b2 = button(screen, (350, 700), "+")
b3 = button(screen, (110, 700), " - ")

L11 = 1.
L12 = 0.84
L13 = 0.92

L21 = 1.08
L22 = 1.04
L23 = 1.

L31 = 0.9
L32 = 0.9
L33 = 1.

L41 = 0.64
L42 = 0.6
L43 = 0.84

In [10]:
VS = """
#version 330

in vec3 in_vert;
in vec2 in_uv;

out vec2 fragCoord;

void main() {
    gl_Position = vec4(in_vert, 1.0f);
    fragCoord = in_uv;
}
"""
run = True
up = True
frame = 0
cameraAngle = 0
camAuto = False
speed = 0.01

alpha = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
P = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
def alphas():
  for i in range(4):
          alpha[i][1] = alpha[i][0] +  0.25 + alpha[i][0]
          alpha[i][2] = alpha[i][1] + alpha[i][0] - 1
          alpha[i][0] += 1.
          for j in range(3):
            P[i][j] = "vec3(" + str(L11*np.cos(alpha[i][j])) + "," + str(L11*np.sin(alpha[i][j])) + ",0.)"


errcpt = 0
alphas()

def fs():
  rFS = """
  #version 330
  out vec4 fragColor; 
  in vec2 fragCoord;

  // Constants
  const int MAX_MARCHING_STEPS = 255;
  const float MIN_DIST = 0.5;
  const float MAX_DIST = 200.0;
  const float PRECISION = 0.01;
  const float EPSILON = 0.05;
  const float PI = 3.14159265359;


float satFactor=0.5;
 float movingSpeed=1.0;

  vec3 center1 = vec3(0.,-1.,1.);
  vec3 P11 = vec3(0.,0.,0.);
  vec3 P12 = vec3(0.,0.,0.);
  vec3 P13 = vec3(0.,0.,0.);

  vec3 center2 = vec3(0.,-1.,0.5);
  vec3 P21 = vec3(0.,0.,0.);
  vec3 P22 = vec3(0.,0.,0.);
  vec3 P23 = vec3(0.,0.,0.);

  vec3 center3 = vec3(0.,-1.,0.);
  vec3 P31 = vec3(0.,0.,0.);
  vec3 P32 = vec3(0.,0.,0.);
  vec3 P33 = vec3(0.,0.,0.);

  vec3 center4 = vec3(0.,-1.,-0.5);
  vec3 P41 = vec3(0.,0.,0.);
  vec3 P42 = vec3(0.,0.,0.);
  vec3 P43 = vec3(0.,0.,0.);

  mat3 rotateX(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(1, 0, 0),
          vec3(0, c, -s),
          vec3(0, s, c)
      );
  }

  mat3 rotateY(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(c, 0, s),
          vec3(0, 1, 0),
          vec3(-s, 0, c)
      );
  }

  mat3 rotateZ(float theta) {
      float c = cos(theta);
      float s = sin(theta);
      return mat3(
          vec3(c, -s, 0),
          vec3(s, c, 0),
          vec3(0, 0, 1)
      );
  }

  // Identity matrix.
  mat3 identity() {
      return mat3(
          vec3(1, 0, 0),
          vec3(0, 1, 0),
          vec3(0, 0, 1)
      );
  }

  struct Surface {
      float sd; // signed distance value
      vec3 col; // color
  };
  float sdRoundBox( vec3 p, vec3 b, float r )
  {
    vec3 q = abs(p) - b;
    return length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0) - r;
  }
  Surface sdBox( vec3 p, vec3 b, vec3 offset, vec3 col, mat3 transform)
  {

    p = (p - offset) * transform; // apply transformation matrix
    vec3 q = abs(p) - b;
    float d = length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0);
    return Surface(d, col);
  }

  Surface sdFloor(vec3 p, vec3 col) {
    float d = p.y + 1.;
    return Surface(d, col);
  }

  float smin(float a, float b, float k) {
      float h = clamp(0.5 + 0.5*(a-b)/k, 0.0, 1.0);
      return mix(a, b, h) - k*h*(1.0-h);
  }

  Surface minWithColor(Surface obj1, Surface obj2) {

    return Surface(smin(obj1.sd,obj2.sd, 0.1), obj2.col);
  }

  Surface sdCapsule( vec3 p, vec3 a, vec3 b, float r , vec3 col)
  {
    vec3 pa = p - a, ba = b - a;
    float h = clamp( dot(pa,ba)/dot(ba,ba), 0.0, 1.0 );
    return Surface(length( pa - ba*h ) - r, col);
  }

  Surface sdRoundBox( vec3 p, vec3 b, float r , vec3 col)
{
  vec3 q = abs(p) - b;
  return Surface(length(max(q,0.0)) + min(max(q.x,max(q.y,q.z)),0.0) - r,col);
}

  Surface sdScene2(vec3 p) {
    Surface co = sdCapsule(p, vec3(0.), vec3(1.), 1., vec3(1.,1.,1.));
    return co;
  }
  vec3 color = vec3(0.12,0.1,0.1);

  Surface sdScene(vec3 p) {
    Surface co = sdCapsule(p, center1, P11, 0.2*1.3, color);
    co = minWithColor(co, sdCapsule(p, P11, P12, 0.18*1.3, color));
    co = minWithColor(co, sdCapsule(p, P12, P13, 0.16*1.3, color));
    
    co = minWithColor(co, sdCapsule(p, center2, P21, 0.19*1.3, color));
    co = minWithColor(co, sdCapsule(p, P21, P22, 0.18*1.3, color));
    co = minWithColor(co, sdCapsule(p, P22, P23, 0.175*1.3, color));
    
    co = minWithColor(co, sdCapsule(p, center3, P31, 0.18*1.3, color));
    co = minWithColor(co, sdCapsule(p, P31, P32, 0.17*1.3, color));
    co = minWithColor(co, sdCapsule(p, P32, P33, 0.16*1.3, color));
    
    co = minWithColor(co, sdCapsule(p, center4, P41, 0.16*1.3, color));
    co = minWithColor(co, sdCapsule(p, P41, P42, 0.15*1.3, color));
    co = minWithColor(co, sdCapsule(p, P42, P43, 0.15*1.3, color));
    co = minWithColor(co, sdRoundBox(p + vec3(-0.1,2.1,-0.2), vec3(0.000000001,1.,0.75), 0.3, color));
    return co;
  }

  Surface rayMarch(vec3 ro, vec3 rd, float start, float end) {
    float depth = start;
    Surface co; // closest object

    for (int i = 0; i < MAX_MARCHING_STEPS; i++) {
      vec3 p = ro + depth * rd;
      co = sdScene(p);
      depth += co.sd;
      if (co.sd < PRECISION || depth > end) break;
    }
    
    co.sd = depth;
    
    return co;
  }

  vec3 calcNormal(in vec3 p) {
      vec2 e = vec2(1, -1) * EPSILON;
      return normalize(
        e.xyy * sdScene(p + e.xyy).sd +
        e.yyx * sdScene(p + e.yyx).sd +
        e.yxy * sdScene(p + e.yxy).sd +
        e.xxx * sdScene(p + e.xxx).sd);
  }


  mat3 camera(vec3 cameraPos, vec3 lookAtPoint) {
    vec3 cd = normalize(lookAtPoint - cameraPos); // camera direction
    vec3 cr = normalize(cross(vec3(0, 1, 0), cd)); // camera right
    vec3 cu = normalize(cross(cd, cr)); // camera up
    
    return mat3(-cr, cu, -cd);
  }

  void main()
  {

  P11 = """ + str(P[0][0]) + """ + center1;
  P12 = """ + str(P[0][1]) + """ + P11;
  P13 = """ + str(P[0][2]) + """ + P12;


  P21 = """ + str(P[1][0]) + """ + center2;
  P22 = """ + str(P[1][1]) + """ + P21;
  P23 = """ + str(P[1][2]) + """ + P22;

  P31 = """ + str(P[2][0]) + """ + center3;
  P32 = """ + str(P[2][1]) + """ + P31;
  P33 = """ + str(P[2][2]) + """ + P32;

  P41 = """ + str(P[3][0]) + """ + center4;
  P42 = """ + str(P[3][1]) + """ + P41;
  P43 = """ + str(P[3][2]) + """ + P42;
    vec2 uv = fragCoord;

    vec3 backgroundColor1 = vec3(0.8 ,0.8,0.83);

    vec3 col = vec3(0.);
    vec3 lp = vec3(""" + str(np.sin(cameraAngle)) + """, 0., """+ str(np.cos(cameraAngle))+"""); // lookat point (aka camera target)
    vec3 ro = vec3(0., 0., 0.); // ray origin that represents camera position
    
    float cameraRadius = 3.25;
    ro.x = cameraRadius * (lp.x + ro.x); // convert to polar 
    ro.z = cameraRadius * (lp.z + ro.z);
    ro.y = cameraRadius * (lp.y + ro.y);
    
    vec3 rd = camera(ro, lp) * normalize(vec3(uv, -1)); // ray direction

    Surface co = rayMarch(ro, rd, MIN_DIST, MAX_DIST); // closest object

    if (co.sd > MAX_DIST) {
      col =  backgroundColor1;
    } else {
      vec3 p = ro + rd * co.sd;
      vec3 normal = calcNormal(p);
      vec3 lightPosition = vec3(-1., 3., 0.);
      vec3 lightDirection = normalize(lightPosition - p);
      vec3 lightPosition3 = vec3(0., -1., 0.);
      vec3 lightDirection3 = normalize(lightPosition - p);
      vec3 lightPosition2 = vec3(-0, -4, 1);
      vec3 lightDirection2 = normalize(lightPosition2 - p);

      float dif = clamp(dot(normal, lightDirection), 0.7, 100.);
      float dif2 = clamp(dot(normal, lightDirection2), 0.7, 100.);
      float dif3 = clamp(dot(normal, lightDirection3), 0.7, 200.);

      col = ((3*dif3) * (2* dif) * (2 * dif2) * (co.col))/2;
      //col = (dif * dif2 * (co.col));
      //col = dif2 * co.col;
      //col = co.col;
    }

    fragColor = vec4(col, 1.0);
  }

  """
  return rFS
FS = fs()
prog = ctx.program(vertex_shader=VS, fragment_shader=FS)
vao = ctx.vertex_array(prog, [(vbo, "3f 2f", "in_vert", "in_uv")], ibo)
vao.render(mode=mgl.TRIANGLES)
camModified = False
while run:
  camModified = False
  calculated = False
  if camAuto == True:
    cameraAngle += speed
    camModified = True
  frame += 1
  clock.tick(0)
  screen.blit(update_fps(), (810,800))
  text = font2.render("ExoTool Viewer V1.2", 1, pygame.Color("orange"))
  screen.blit(text, (10,20))
  try:
    if (ser.inWaiting() > 0):
      strval = ser.readline().decode().strip()
      if "A" in strval:
        alpha[0][0] = ((int(strval.split("A")[1].split("B")[0])*1.6)/1024)-0.85
        alpha[1][0] = ((int(strval.split("B")[1].split("C")[0])*1.6)/1024)-0.85
        alpha[2][0] = ((int(strval.split("C")[1].split("D")[0])*1.6)/1024)-0.85
        alpha[3][0] = ((int(strval.split("D")[1])*1.6)/1024)-0.85
        alphas()
        #start = time.time()
        FS = fs()
        prog = ctx.program(vertex_shader=VS, fragment_shader=FS)
        vao = ctx.vertex_array(prog, [(vbo, "3f 2f", "in_vert", "in_uv")], ibo)
        vao.render(mode=mgl.TRIANGLES)
        #end = time.time()
        #print((end - start) * 1000)
    errcpt = 0
    calculated = True
  except Exception:
          errcpt += 1
          if (errcpt >= 5):
               print("Connection error, please check cables")
               text = font2.render("waiting for connection...", 1, pygame.Color("red"))
               screen.blit(text, (1200, 10))
               pygame.display.update()
               found = False
               while found == False:
                    time.sleep(1)
                    try:
                         ser = portsDetection()
                         found = True
                    except Exception:
                         found = False
          pass
  try:
    if (camModified == True and calculated == False):
      FS = fs()
      prog = ctx.program(vertex_shader=VS, fragment_shader=FS)
      vao = ctx.vertex_array(prog, [(vbo, "3f 2f", "in_vert", "in_uv")], ibo)
      vao.render(mode=mgl.TRIANGLES)
  except Exception:
    print("error while updating camera angle")
  pygame.display.update()

  for event in pygame.event.get():
      if event.type == pygame.QUIT:
        run = False
      if event.type == pygame.MOUSEBUTTONDOWN:
        if b1.collidepoint(pygame.mouse.get_pos()):
          camAuto = True
        if b2.collidepoint(pygame.mouse.get_pos()):
          camAuto = False
          cameraAngle += 0.2
        if b3.collidepoint(pygame.mouse.get_pos()):
          camAuto = False
          cameraAngle -= 0.2

  #ctx.clear(0.0, 0.0, 0.0, 1.0)
  show_fbo(fbo, SIZE, COLOR_MODE)
  #pygame.display.flip()
  if frame>30 and frame%30 == 0:
    fig = plt.figure(figsize=(8.4, 2.0), dpi=100)
    ax = fig.add_subplot(1, 1, 1)
    end = time.time()
    ax.plot(range(len(fpslist))[-600:-1],fpslist[-600:-1])
    fig.canvas.draw()
    pil = Image.frombytes('RGB',fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    surf = pilImageToSurface(pil)
    matplotlib.pyplot.close()
  try:
    screen.blit(surf, surf.get_rect(center = (430, 900)))
  except Exception:
    pass
pygame.quit()

Connection error, please check cables
waiting for connection...
Arduino (www.arduino.cc)
This is an Arduino!
